# Path e librerie
## Prima di modificare avvertire gli altri!

## Install

In [ ]:
#import drive dir
from google.colab import drive
drive_dir = 'drive' ; 
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install polyglot
!pip install PyICU
!pip install pycld2
!pip install morfessor

     |████████████████████████████████| 126 kB 36.1 MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52578 sha256=c68973e90de9d5f9b15413528324cf6d21c8a0412fb9b18a2e1eaba207d3672f
  Stored in directory: /root/.cache/pip/wheels/09/bc/67/75c9de8e9726460bc0b101ad225ad025cb8ce9e0759beb9d52
Successfully built polyglot
     |████████████████████████████████| 299 kB 32.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for PyICU: filename=PyICU-2.8-cp37-cp37m-linux_x86_64.whl size=1377130 sha256=bdf61be8cbae56ae2a9edd0b60b63d86986055d1ff951949689dfe8758539d5f
  Stored in directory: /root/.cache/pip/wheels/14/bd/45/aeddc643bd0637c14fa27bffaee5b411cdc323f8bec76ad15e
Successfully built PyICU
     |████████████████████████████████| 41.4 MB 1.2 MB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-linux_x86_64.whl size=9834313 sha256=6ae377d465a1c69830

## Import

In [ ]:
import tweepy
import numpy as np
from tweepy import Stream, OAuthHandler
from tqdm.notebook import tqdm
# from tqdm import tqdm !! per Jupyter from tqdm.notebook non funziona, runnare questo
import pandas as pd
pd.set_option('display.max_colwidth', None) # setta il num. di caratteri visibili per ogni cella della colonna
# es. 50 = primi 50 caratteri
# None = testo completo

import io
import pprint
import os
import glob
import warnings
warnings.filterwarnings('ignore')

from polyglot.detect import Detector
import icu
import csv
import re

## Path per Colab

In [ ]:
##########################
### ROOT
##########################

prj_root_dir = f'/content/{drive_dir}/MyDrive/nucleare/'
os.makedirs(prj_root_dir, exist_ok=True)

##########################
### SCRAPING: file dello scraping di ATP
##########################

scrapingATP_root_dir = prj_root_dir + 'scrapingATP/'
os.makedirs(scrapingATP_root_dir, exist_ok=True)

##########################
### ETL: 
##########################

etl_root_dir = prj_root_dir + 'ETL/'
os.makedirs(etl_root_dir, exist_ok=True)

##########################
### DWH
##########################

dw_root_dir = prj_root_dir + 'DWH/'
os.makedirs(dw_root_dir, exist_ok=True)

##########################
### SENTIMENT
##########################

sentiment_root_dir = prj_root_dir + 'SENTIMENT/'
os.makedirs(sentiment_root_dir, exist_ok=True)

##########################
### WORDEMBEDDING
##########################

embedding_root_dir = prj_root_dir + 'EMBEDDING/'
os.makedirs(embedding_root_dir, exist_ok=True)

##########################
### OUTPUT
##########################

output_root_dir = prj_root_dir + 'OUTPUT/'
os.makedirs(output_root_dir, exist_ok=True)

##########################
### MODELS
##########################

models_root_dir = prj_root_dir + 'MODELS/'
os.makedirs(models_root_dir, exist_ok=True)

##########################
### STAGING ANTONELLO M.
##########################

st_man_root_dir = prj_root_dir + 'STAGING_MANENTI/'
os.makedirs(st_man_root_dir, exist_ok=True)


##########################
### STAGING TONE
##########################

st_tone_root_dir = prj_root_dir + 'STAGING_TONE/'
os.makedirs(st_tone_root_dir, exist_ok=True)

##########################
### STAGING MARCO
##########################

st_marco_root_dir = prj_root_dir + 'STAGING_MARCO/'
os.makedirs(st_marco_root_dir, exist_ok=True)

##########################
### STAGING NATALIA
##########################

st_natalia_root_dir = prj_root_dir + 'STAGING_NATALIA/'
os.makedirs(st_natalia_root_dir, exist_ok=True)

### Path per Jupyter
Solo quando si usa Jupyter runnare, il percorso dovrebbe essere lo stesso per tutti/e

In [ ]:
##########################
### ROOT
##########################

drive_dir = f'G:\\My Drive'
prj_root_dir = f'{drive_dir}\\nucleare\\'

##########################
### SCRAPING
##########################

scrapingATP_root_dir = prj_root_dir + 'scrapingATP\\'

##########################
### ETL
##########################

etl_root_dir = prj_root_dir + 'ETL\\'

##########################
### DWH
##########################

dw_root_dir = prj_root_dir + 'DWH\\'

##########################
### SENTIMENT
##########################

sentiment_root_dir = prj_root_dir + 'SENTIMENT\\'

##########################
### WORDEMBEDDING
##########################

embedding_root_dir = prj_root_dir + 'EMBEDDING\\'

##########################
### OUTPUT
##########################

output_root_dir = prj_root_dir + 'OUTPUT\\'

##########################
### STAGING ANTONELLO M.
##########################

st_man_root_dir = prj_root_dir + 'STAGING_MANENTI\\'

# 1. Ingest
## Antonello

cartella di scraping: nulceare/scrapingATP

## Codici Twitter

In [ ]:
consumer_key = 'WYOB4KubrmdOM9ycdyGvBLkTa'# API Key
consumer_secret = 'AThpoW6fKDm6bwZYeO0C0AVM7jKVq0bVgU7w9VfMejrnkygBTK' # APY Secret Key
access_token = '1396896479525097474-2HgbagRtx3ByeO4zmkWSPaL1iOxEiH'
access_secret = 'IKvhdjwGMDky2dAN6OYdeMZhWUHYheqTmMEWdgcInXf4F'
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

# consumer_key = 'MQXChIn29dCky3pOQyW890Uxi'
# consumer_secret = 'u4n3UeFz58b4H6InWFhBDUPZsfJ8xdVS0wODdnfAjz6e6PUbCJ'
# access_token = '262247836-oWkm6AZIw4K0Fldnt8VBXwuL9IbtNCZwVj73ODkI'
# access_secret = 'GySGl7BGaYvmXUtCapu5pHwEDHD5tA3Qd4yCMIDPKm1Wx'
# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)
# api = tweepy.API(auth, wait_on_rate_limit=True)

# consumer_key = 'Vzk6O5t967f1n4iz9iQcsQwhf'
# consumer_secret = 'Q8sncdY1j85mwNitOqb5Spcg6U6uuZ0E2KNHHZEYF2J4DdCyVi'
# access_token = '164224789-mFmU5YhessCNvubNojgfL605tbU0vh0NOna3HdQU'
# access_secret = 'Ic5LNOH2QqxziSq4eTQdJvbXo3noTcQDUzCKXgFcdPzpC'
# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)
# api = tweepy.API(auth, wait_on_rate_limit=True)

# consumer_key = 'A4GRiWdUDgiGPckQSykzA'
# consumer_secret = 'nSyjpFDTr3eEmd8tFeBM1lwVrTNPLaHrcx0DicTEEuw'
# access_token = '1678747572-lbkg1jxpe5HypoTRSnRMeiVIe0RBMxhdjLJ0GJ6'
# access_secret = '1qVP0pHXV9HDgKhHDIy2HHqqnHFBm3sI5AR2ZDwZxlM'
# auth = OAuthHandler(consumer_key, consumer_secret)
# auth.set_access_token(access_token, access_secret)
# api = tweepy.API(auth, wait_on_rate_limit=True)

In [ ]:
utente = 'antonello' #mettere il proprio nome
lang = 'all' #mettere la lingua
prog = '20' #mettere il progressivo

## Scraping dello storico

In [ ]:
#mettere la query
query_00 = ('NittoATPFinals' or 'TennisTakesOverTorino' or '#ATPFinals' or 'ATPFinals' or 'ATP Finals' or 'Norrie' or '#Norrie' or '@cam_norrie' or 'Norrie' or '#Norrie' or '@cam_norrie' or 'Ruud' or '#Ruud' or '@CasperRuud98' or 'Rublev' or '#Rublev' or '@AndreyRublev97' or 'Tsitsipas' or '#Tsitsipas' or '@steftsitsipas' or 'Djokovic' or '#Djokovic' or '@DjokerNole' or 'Sinner' or '#Sinner' or '@janniksin' or 'Medvedev' or '#Medvedev' or '@DaniilMedwed' or 'Zverev' or '#Zverev' or '@AlexZverev' 'Berrettini' or '#Berrettini' or '@MattBerrettini' or 'Hurkacz' or '#Hurkacz' or '@HubertHurkacz')
query_01 = ('@DaniilMedwed' or 'Medvedev' or '#Medvedev' or '@AlexZverev' or 'Zverev' or '#Zverev' or '#NittoATPFinals' or 'NittoATPFinals' or 'TennisTakesOverTorino' or '#ATPFinals' or 'ATPFinals' or 'ATP Finals')

In [ ]:
searched_tweets = []
last_id = -1
max_tweets = 500_000
pbar = tqdm(total=max_tweets)

while len(searched_tweets) < max_tweets:
    count = max_tweets - len(searched_tweets)
    try:
        # api.search è un metodo che su colab funziona anche se non è documentato in Tweepy. Su Jupyter non funziona, sostituire con search_tweets
        new_tweets = api.search(q=query_01, count=1000, max_id=str(last_id - 1), tweet_mode="extended") # tweet_mode="extended" importante, altrimenti il testo del tweet viene troncato
        # If I've reached the end of the search then I'm done
        # immediately the cycle while
        if not new_tweets:
            break
        # added the data found to the list
        searched_tweets.extend(new_tweets)
        # tqdm
        pbar.update(len(new_tweets))
        # retrieve the last id found
        last_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        print(e)
        break

pbar.close()

In [ ]:
print('Totale Tweet')
len(searched_tweets)

In [ ]:
dict_ = {'id': [], 'user': [], 'date': [], 'text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}

for status_j in searched_tweets[0:max_tweets]:
    status = status_j._json

    if 'retweeted_status' in status:
      if 'extended_tweet' in status['retweeted_status']:
        text = status['retweeted_status']['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['retweeted_status']['full_text']
        dict_['text'].append(text)
    else:
      if 'extended_tweet' in status:
        text = status['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['full_text']
        dict_['text'].append(text)
    
    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])

df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by=['date'], inplace=True, ascending=True)
df.head(10)

In [ ]:
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by=['date'], inplace=True, ascending=True)
df.head(10)

In [ ]:
df.info()

In [ ]:
print(df.shape)

In [ ]:
df.to_csv(f'{scrapingATP_root_dir}twitter_history_{utente}_{prog}_{lang}_{len(searched_tweets)}.csv')

## Scraping dello stream

In [ ]:
class MyStreamListener(tweepy.StreamListener):
    data = [] #qui ci metto i tweet che scarico man mano
    num = 0 #questo è un contatore settato a 0

    def store(self):
      dict_ = {'id': [], 'user': [], 'date': [], 'text': [],
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
      
      for status_j in self.data:
          status = status_j._json

          if 'retweeted_status' in status:
            if 'extended_tweet' in status['retweeted_status']:
              text = status['retweeted_status']['extended_tweet']['full_text']
              dict_['text'].append(text)
            else:
              text = status['retweeted_status']['full_text']
              dict_['text'].append(text)
          else:
            if 'extended_tweet' in status:
              text = status['extended_tweet']['full_text']
              dict_['text'].append(text)
            else:
              text = status['full_text']
              dict_['text'].append(text)

          dict_['id'].append(status['id'])
          dict_['user'].append(status['user']['screen_name'])
          dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
          dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
          dict_['location'].append(status['user']['location'])
          dict_['followers_count'].append(status['user']['followers_count'])
          dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
          dict_['date'].append(status['created_at'])
          dict_['full_text'].append(status['full_text'])
          dict_['favorite_count'].append(status['favorite_count'])
          dict_['retweet_count'].append(status['retweet_count'])
          dict_['retweet'].append(status['retweeted'])

      df = pd.DataFrame.from_dict(dict_, orient="columns")
      df = df.set_index("id")
      df.sort_values(by='favorite_count', inplace=True, ascending=False)
      df.to_csv(f'{scrapingATP_root_dir}_twitter_stream_{utente}_{prog}_{lang}_' + str(self.num) + '.csv') #salvo in csv
      self.num = self.num + 1
      self.data = []


    def on_status(self, status):
        self.data.append(status)
        # to pandas
        # store su google drive
        if len(self.data) > 5000:
          self.store()

In [ ]:
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener, tweet_mode="extended")

In [ ]:
query_00 = ('NittoATPFinals' or 'TennisTakesOverTorino' or '#ATPFinals' or 'ATPFinals' or 'ATP Finals' or 'Norrie' or '#Norrie' or '@cam_norrie' or 'Norrie' or '#Norrie' or '@cam_norrie' or 'Ruud' or '#Ruud' or '@CasperRuud98' or 'Rublev' or '#Rublev' or '@AndreyRublev97' or 'Tsitsipas' or '#Tsitsipas' or '@steftsitsipas' or 'Djokovic' or '#Djokovic' or '@DjokerNole' or 'Sinner' or '#Sinner' or '@janniksin' or 'Medvedev' or '#Medvedev' or '@DaniilMedwed' or 'Zverev' or '#Zverev' or '@AlexZverev' 'Berrettini' or '#Berrettini' or '@MattBerrettini' or 'Hurkacz' or '#Hurkacz' or '@HubertHurkacz')
query_01 = ['@DaniilMedwed' or 'Medvedev' or '#Medvedev' or '@AlexZverev' or 'Zverev' or '#Zverev' or '#NittoATPFinals' or 'NittoATPFinals' or 'TennisTakesOverTorino' or '#ATPFinals' or 'ATPFinals' or 'ATP Finals']
myStream.filter(track=query_01)

In [ ]:
len(myStreamListener.data)

In [ ]:
dict_ = {'id': [], 'user': [], 'date': [], 'text': [], 
         'favorite_count': [], 'hashtags': [], 
         'location': [], 'retweet': [], 'retweet_count': [], 'followers_count': [], 'in_reply_to_status_id':[], 'user_mentions':[]}
#
for status_j in myStreamListener.data:
    status = status_j._json

    if 'retweeted_status' in status:
      if 'extended_tweet' in status['retweeted_status']:
        text = status['retweeted_status']['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['retweeted_status']['text']
        dict_['text'].append(text)
    else:
      if 'extended_tweet' in status:
        text = status['extended_tweet']['full_text']
        dict_['text'].append(text)
      else:
        text = status['text']
        dict_['text'].append(text)

    dict_['id'].append(status['id'])
    dict_['user'].append(status['user']['screen_name'])
    dict_['hashtags'].append([hashtag['text'] for hashtag in status['entities']['hashtags']])
    dict_['user_mentions'].append([user_mention['screen_name'] for user_mention in status['entities']['user_mentions']])
    dict_['location'].append(status['user']['location'])
    dict_['followers_count'].append(status['user']['followers_count'])
    dict_['in_reply_to_status_id'].append(status['in_reply_to_status_id'])
    dict_['date'].append(status['created_at'])
    dict_['favorite_count'].append(status['favorite_count'])
    dict_['retweet_count'].append(status['retweet_count'])
    dict_['retweet'].append(status['retweeted'])

In [ ]:
df = pd.DataFrame.from_dict(dict_, orient="columns")
df = df.set_index("id")
df.sort_values(by='favorite_count', inplace=True, ascending=False)
df.head(10)

In [ ]:
df.to_csv(f'{scrapingATP_root_dir}twitter_stream_{utente}_{prog}_{lang}_{len(myStreamListener.data)}.csv') #scarico l'ultimo stream

In [ ]:
#unisco tutti i csv

path = scrapingATP_root_dir

all_files = glob.glob(os.path.join(path, "twitter_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged = pd.concat(df_from_each_file, ignore_index=True)
df_merged.drop_duplicates(subset=['id'], inplace=True, ignore_index=False)
df_merged.to_csv(f'{etl_root_dir}twitter_merged_from_scraping.csv', index=False, sep=',', encoding='utf-8')

In [ ]:
df_merge = pd.read_csv(f'{etl_root_dir}twitter_merged_from_scraping.csv')
df_merge.info()

In [ ]:
df_merge.tail()

# 2. ETL
**NATALIA**



1.   rimuovere #
2.   rimuovere id doppi
1.   rimuovere http
2.   rimuovere twitter con testo vuoto
1.   rimuovere @



file input: nucleare/ETL/twitter_merged_from_scraping.csv

cartella output: nucleare/DWH/

In [ ]:
df = pd.read_csv(etl_root_dir+'twitter_merged_from_scraping.csv')

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
with open(dw_root_dir+'twitter_text_cleaned.csv', mode='w') as csv_file_out:
  csv_writer_out = csv.writer(csv_file_out, delimiter=',', quotechar='"')
  with open(etl_root_dir+'twitter_merged_from_scraping.csv', mode='r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    max_lines=10000000 
    monDict = {'Gen' : '01' , 'Feb' : '02', 'Mar' : '03', 'Apr' : '04', 'May' : '05', 'Jun' : '06', 'Jul' : '07', 'Aug' : '08', 'Sep' : '09', 'Oct' : '10', 'Nov' : '11', 'Dic' : '12'}
  
    for row in csv_reader:
          if line_count > 0:
              dateCur=row[2]
              monCur=monDict[dateCur[4:7]]
              dateNew=dateCur[26:30]+'-'+monDict[dateCur[4:7]]+'-'+dateCur[8:10]+' '+dateCur[11:13]+':'+dateCur[14:16]+':'+dateCur[17:19]
              timezone='z'+dateCur[20:25]
              #print (f'{dateCur} ==> {dateNew}')
              textnonl=row[3].replace("\n"," ")
              textnonl==re.sub(r"\s+"," ",textnonl) # tolgo doppi spazi
              #print (f'message {message}')
              textnonlhash=re.sub(r"#\w+ ?","",textnonl)
              textnonlhash=re.sub(r"\s+"," ",textnonlhash)
              textnonlhashmen=re.sub(r"@\w+ ?","",textnonlhash)
              textnonlhashmen=re.sub(r"\s+"," ",textnonlhashmen)
              textnonlhashmenhttp=re.sub(r"http\S+","",textnonlhashmen)
              textnonlhashmenhttp=re.sub(r"\s+"," ",textnonlhashmenhttp)
              textnonlhashmenhttpnum=re.sub(r"\d+", "", textnonlhashmenhttp)    
              textnonlhashmenhttpnum=re.sub(r"\s+"," ",textnonlhashmenhttpnum)
              textnonlhashmenhttpnumsimb=re.sub(r"[^\w\s]"," ",textnonlhashmenhttpnum)
              textnonlhashmenhttpnumsimb=re.sub(r"\s+"," ",textnonlhashmenhttpnumsimb)
              row2=row
              row2.extend([dateNew,timezone,textnonl,textnonlhash,textnonlhashmen,textnonlhashmenhttp,textnonlhashmenhttpnum,textnonlhashmenhttpnumsimb])
              csv_writer_out.writerow(row2)
              line_count += 1
          else:
            row2=row
            row2.extend(['date_norm','time_zone','text_nonl','text_nonl_hash','text_nonl_hash_men','text_nonl_hash_men_http','text_nonl_hash_men_http_num','text_nonl_hash_men_http_num_simb'])
            csv_writer_out.writerow(row2)
            line_count += 1
          if line_count > max_lines:
              print (f'Exit')
              break      
    print(f'Processed {line_count} lines.')

In [ ]:
df = pd.read_csv(dw_root_dir+'twitter_text_cleaned.csv', sep=',')
df.info()

# 3. Language detection
ANTONELLO M.

In [ ]:
df = pd.read_csv(f'{dw_root_dir}twitter_text_cleaned.csv')
df.info()

In [ ]:
df.sample(n=5, random_state=13)

In [ ]:
# testing polyglot
df_test = df[['text_nonl_hash_men_http_num_simb', 'text']]
df_test = df_test.sample(n=3, random_state=13)
df_test

In [ ]:
df_test['poly_obj'] = df_test.text_nonl_hash_men_http_num_simb.apply(lambda x: Detector(x, quiet=True))
df_test['text_lang'] = df_test.poly_obj.apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
df_test['text_lan_code'] = df_test.poly_obj.apply( lambda x: x.language.code)
df_test['lang_confidence'] = df_test.poly_obj.apply( lambda x: x.language.confidence)

In [ ]:
df_test.info()

In [ ]:
df_test

In [ ]:
df_test['poly_obj'] = df_test['poly_obj'].astype(str)
df_test_predict = df_test['poly_obj'].str.split(" ", expand=True)
df_test_predict.rename(columns={3:'prediction'}, inplace=True)
df_test_predict = df_test_predict.drop([0, 1, 2, 4, 5, 6], axis=1)
df_test_predict = df_test_predict.iloc[:,0:1]
df_test_predict['prediction_reliable'] = df_test_predict['prediction'].str.replace(r'\n([-A-Za-z0-9_.]+)', '') 
df_test_predict = df_test_predict.drop(['prediction'], axis=1)
df_test = pd.concat([df_test, df_test_predict], axis=1) 
df_test = df_test.drop(['poly_obj'], axis=1)
df_test

In [ ]:
# apply polyglot
df = df[df['text_nonl_hash_men_http_num_simb'].notna()]
df['poly_obj'] = df['text_nonl_hash_men_http_num_simb'].apply(lambda x: Detector(x, quiet=True))
df['text_lang'] = df.poly_obj.apply(lambda x: icu.Locale.getDisplayName(x.language.locale))
df['text_lan_code'] = df.poly_obj.apply( lambda x: x.language.code)
df['lang_confidence'] = df.poly_obj.apply( lambda x: x.language.confidence)

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df['poly_obj'] = df['poly_obj'].astype(str)
df_det_lang = df['poly_obj'].str.split(" ", expand=True)
df_det_lang.rename(columns={3:'prediction'}, inplace=True)
df_det_lang = df_det_lang.drop([0, 1, 2, 4, 5, 6], axis=1)
df_det_lang = df_det_lang.iloc[:,0:1]
df_det_lang['prediction_reliable'] = df_det_lang['prediction'].str.replace(r'\n([-A-Za-z0-9_.]+)', '') 
df_det_lang = df_det_lang.drop(['prediction'], axis=1)
df = pd.concat([df, df_det_lang], axis=1) 
df = df.drop(['poly_obj'], axis=1)
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
twe = df['id'].count()
lan = df['text_lang'].nunique()
print(f'{lan} lingue differenti per {twe} tweet di cui:')
df.groupby(['text_lang']).size().nlargest(10).reset_index(name='top10')

In [ ]:
df.to_csv(f'{st_man_root_dir}twitter_text_cleaned_lang_det.csv', index=False)

# 4. Player detection
ANTONELLO M.

In [ ]:
df = pd.read_csv(f'{st_man_root_dir}twitter_text_cleaned_lang_det.csv')

In [ ]:
df.info()

**Idea**
1. Si creano tante colonne quante sono le variabili in list (cella iniziale)
2. Se una delle parole definite nella lista (.str.contains) è presente si mette 1 colonna corrispondente corrispondete.
3. Se la somma delle nuove colonne è 0 il topic è Other
4. Se la somma delle nuove colonne è 1 il topic è il nome della colonna con valore 1.
5. Se la somma delle nuove colonne è 2 e ATP Finals = 1 allora il topic è il nome dell'altra colonna
6. Se la somma delle nuove colonne è 2 e ATP Finals = 0 allora il topic è il nome previsto da match se le colonne corrispettive sono a 1, altrimenti il topic è il nome di entrambi i giocatori.
7. Se la somma delle nuove colonne è 3 e ATP Finals = 1 allora il topic è il nome previsto da match se le colonne corrispettive sono a 1, altrimenti il topic è il nome di tutti i giocatori = 1.
8. Se la somma delle nuove colonne è 3 e ATP Finals = 0 il topic sono tutti i nomi dei giocatori = 1.
9. Se la somma delle nuove colonne è > 3 il topic è il nome di tutti i giocatori = 1


**Match**

Djokovic and Ruud = 'Djokovic VS Ruud RR' # RR is Round Robin cioè fase a gironi

Rublev and Tsitsipas = 'Rublev VS Tsitsipas RR'

Djokovic and Rublev = 'Djokovic VS Rublev RR'

Ruud and Norrie = 'Ruud VS Norrie RR'

Ruud and Rublev = 'Ruud VS Rublev RR'

Djokovic and Norrie = 'Djokovic VS Norrie RR'

Medvedev and Hurkacz = 'Medvedev VS Hurkacz RR'

Zverev and Berrettini = 'Zverev VS Berrettini RR'

Medvedev and Zverev = 'Medvedev VS Zverev RR'

Sinner and Hurkacz = 'Sinner VS Hurkacz RR'

Zverev and Hurkacz = 'Zverev VS Hurkacz RR'

Medvedev and Sinner = 'Medvedev VS Sinner RR'

Djokovic and Zverev = 'Djokovic VS Zverev SF' # SF is Semi-Final

Medvedev and Ruud = 'Medvedev VS Ruud SF'

Medvedev and Zverev = 'Medvedev VS Zverev F' #F is Final

In [ ]:
# inizio definendo per ogni topic le cose che deve contenere
Medvedev = ['Medvedev', '#Medvedev', '@DaniilMedwed']
Zverev = ['Zverev', '#Zverev', '@AlexZverev']
Berrettini = ['Berrettini', '#Berrettini', '@MattBerrettini']
Hurkacz = ['Hurkacz', '#Hurkacz', '@HubertHurkacz']
Sinner = ['Sinner', '#Sinner', '@janniksin']
Djokovic = ['Djokovic', '#Djokovic', '@DjokerNole']
Tsitsipas = ['Tsitsipas', '#Tsitsipas', '@steftsitsipas']
Rublev = ['Rublev', '#Rublev', '@AndreyRublev97']
Ruud = ['Ruud', '#Ruud', '@CasperRuud98']
Norrie = ['Norrie', '#Norrie', '@cam_norrie']
ATP_Finals = ['NittoATPFinals', 'TennisTakesOverTorino', '#ATPFinals', 'ATPFinals']

In [ ]:
# creo le colonne con valori 1 e 0
df['Medvedev'] = np.where(df['text'].str.contains('|'.join(Medvedev)), 1, 0)
df['Zverev'] = np.where(df['text'].str.contains('|'.join(Zverev)), 1, 0)
df['Berrettini'] = np.where(df['text'].str.contains('|'.join(Berrettini)), 1, 0)
df['Hurkacz'] = np.where(df['text'].str.contains('|'.join(Hurkacz)), 1, 0)
df['Sinner'] = np.where(df['text'].str.contains('|'.join(Sinner)), 1, 0)
df['Djokovic'] = np.where(df['text'].str.contains('|'.join(Djokovic)), 1, 0)
df['Tsitsipas'] = np.where(df['text'].str.contains('|'.join(Tsitsipas)), 1, 0)
df['Rublev'] = np.where(df['text'].str.contains('|'.join(Rublev)), 1, 0)
df['Ruud'] = np.where(df['text'].str.contains('|'.join(Ruud)), 1, 0)
df['Norrie'] = np.where(df['text'].str.contains('|'.join(Norrie)), 1, 0)
df['ATP_Finals'] = np.where(df['text'].str.contains('|'.join(ATP_Finals)), 1, 0)

In [ ]:
df_topic = df[["ATP_Finals", "Sinner", "Norrie", "Ruud", "Rublev", "Tsitsipas", "Medvedev", "Zverev", "Berrettini", "Hurkacz", "Djokovic"]]
df_topic_NO_ATP = df[["Sinner", "Norrie", "Ruud", "Rublev", "Tsitsipas", "Medvedev", "Zverev", "Berrettini", "Hurkacz", "Djokovic"]]

df['topic'] = df_topic_NO_ATP.apply(lambda row: row[row == 1].index.values, axis=1)

In [ ]:
# df = df.drop(['ATP_Finals', 'Norrie', 'Ruud', 'Medvedev', 'Zverev', 'Berrettini', 'Hurkacz', 'Sinner', 'Djokovic', 'Tsitsipas', 'Rublev'], axis=1)
df.to_csv(f'{st_man_root_dir}twitter_text_cleaned_lang_and_player_det.csv', index=False, sep=',', encoding='utf-8')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
# per staging Marco 2 subset: it e en
df_en = df[df["text_lang"].str.contains("English")]
df_it = df[df["text_lang"].str.contains("Italian")]

In [ ]:
print(df_en.shape)
print(df_it.shape)

In [ ]:
df_en.info()

In [ ]:
df_it.info()

In [ ]:
df_en.head()

In [ ]:
df_it.head()

In [ ]:
# to stagin Ant M.
df_en.to_csv(f'{st_man_root_dir}twitter_text_cleaned_lang_and_player_det_EN.csv', index=False, sep=',', encoding='utf-8')
df_it.to_csv(f'{st_man_root_dir}twitter_text_cleaned_lang_and_player_det_IT.csv', index=False, sep=',', encoding='utf-8')

# to staging Marco
df_en.to_csv(f'{st_marco_root_dir}twitter_text_cleaned_lang_and_player_det_EN.csv', index=False, sep=',', encoding='utf-8')
df_it.to_csv(f'{st_marco_root_dir}twitter_text_cleaned_lang_and_player_det_IT.csv', index=False, sep=',', encoding='utf-8')

## Labeling

In [ ]:
# creare prima il subset che interessa e lo salvo per salvare il seme
# una volta fatto commentare queste celle
# ATTENZIONE già fatto, lasciare commentate!

# df = pd.read_csv(f'{st_man_root_dir}twitter_text_cleaned_lang_and_player_det_IT.csv')
# df_label = df.sample(n=2000) #frac=0.7
# df_label.to_csv(f'{st_man_root_dir}sample_2000_for_labeling_IT.csv', index=False, sep=',', encoding='utf-8')

In [ ]:
# riprendo da qui il sample generato sopra
df_label = pd.read_csv(f'{st_man_root_dir}sample_2000_for_labeling_IT.csv')

In [ ]:
df_label.shape

In [ ]:
# divido il df in pezzi da 100 e procedo in ordine
label_100 = df_label.iloc[:100,:] # fatto
label_200 = df_label.iloc[100:200,:] # fatto
label_300 = df_label.iloc[200:300,:] # fatto
label_400 = df_label.iloc[300:400,:] # fatto
label_500 = df_label.iloc[400:500,:] # fatto
label_600 = df_label.iloc[500:600,:] # fatto
label_700 = df_label.iloc[600:700,:] # fatto
label_800 = df_label.iloc[700:800,:] # fatto
label_900 = df_label.iloc[800:900,:] # fatto
label_1000 = df_label.iloc[900:1000,:] # fatto
label_1100 = df_label.iloc[1000:1100,:] # fatto
label_1200 = df_label.iloc[1100:1200,:]
label_1300 = df_label.iloc[1200:1300,:]
label_1400 = df_label.iloc[1300:1400,:]
label_1500 = df_label.iloc[1400:1500,:]
label_1600 = df_label.iloc[1500:1600,:]
label_1700 = df_label.iloc[1600:1700,:]
label_1800 = df_label.iloc[1700:1800,:]
label_1900 = df_label.iloc[1800:1900,:]
label_2000 = df_label.iloc[1900:2000,:]

In [ ]:
# creo una lista vuota
sentiment = []
count = 1
# cambiare label_ ad ogni giro (1 occorrenza)
for b in tqdm(label_1100.text_nonl_hash_men_http_num):
  print()
  print(count)
  print(b)
  add_sent = input('1=pos, 0=neg, 2=neu: ')
  add_sent = add_sent.strip(' ')
  sentiment.append(add_sent)
  count += 1

# cambiare label_ ad ogni giro (1 occorrenza)
label_1100['sentiment'] = sentiment

In [ ]:
# cambiare label_ ad ogni giro (2 occorrenze)
label_1100.to_csv(f'{st_man_root_dir}label_1100_IT.csv', index=False, sep=',', encoding='utf-8')

In [ ]:
# merge di tutti i label fatti

path = st_man_root_dir

all_files = glob.glob(os.path.join(path, "label_*.csv"))
df_from_each_file = (pd.read_csv(f, sep=',') for f in all_files)
df_merged = pd.concat(df_from_each_file, ignore_index=True)

# per sicurezza
df_merged.drop_duplicates(subset=['id'], inplace=True, ignore_index=False)
df_merged.dropna(subset=['sentiment'], inplace=True)

df_merged.to_csv(f'{st_tone_root_dir}label_IT.csv', index=False, sep=',', encoding='utf-8')
df_merged.to_csv(f'{st_natalia_root_dir}label_IT.csv', index=False, sep=',', encoding='utf-8')

In [ ]:
df_merged.shape

In [ ]:
df_merged.info()

In [ ]:
df_merged.head()

# 5. Sentiment
MARCO

In [ ]:
pip install vaderSentiment

     |████████████████████████████████| 125 kB 15.8 MB/s 


## Importo Dataset

In [13]:
import pandas as pd
dfen = pd.read_csv('/content/drive/MyDrive/nucleare/STAGING_MARCO/Twitter_Manenti_Files/twitter_text_cleaned_lang_and_player_det_EN.csv')
dfen.shape

(49252, 36)

In [15]:
dfit = pd.read_csv('/content/drive/MyDrive/nucleare/STAGING_MARCO/Twitter_Manenti_Files/twitter_text_cleaned_lang_and_player_det_IT.csv')
dfit.shape

(15985, 36)

## Vader En

In [16]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


vader=[]

analyzer = SentimentIntensityAnalyzer()

for x in (dfen.loc[:,'text_nonl_hash_men_http_num']):
  a = analyzer.polarity_scores(x)
  vader.append(a)

In [18]:
df2 = pd.DataFrame(vader)
df2.shape

(49252, 4)

In [21]:
print(df2)
df2.to_csv('/content/drive/MyDrive/nucleare/STAGING_MARCO/Vader_en.csv')

         neg    neu    pos  compound
0      0.145  0.681  0.174    0.1759
1      0.104  0.563  0.333    0.7430
2      0.000  1.000  0.000    0.0000
3      0.104  0.563  0.333    0.7430
4      0.145  0.681  0.174    0.1759
...      ...    ...    ...       ...
49247  0.000  0.740  0.260    0.6784
49248  0.000  1.000  0.000    0.0000
49249  0.062  0.684  0.254    0.7845
49250  0.000  1.000  0.000    0.0000
49251  0.320  0.680  0.000   -0.5093

[49252 rows x 4 columns]


In [41]:
df2 = pd.read_csv('/content/drive/MyDrive/nucleare/STAGING_MARCO/Vader_en.csv')
df2.head(4)

,Unnamed: 0,neg,neu,pos,compound
0,0,0.145,0.681,0.174,0.1759
1,1,0.104,0.563,0.333,0.7430
2,2,0.000,1.000,0.000,0.0000
3,3,0.104,0.563,0.333,0.7430


## MERGE -- csv_tradotti

In [24]:
import os
import glob
import pandas as pd
os.chdir("/content/drive/MyDrive/nucleare/STAGING_MARCO/PYTHON_TRADOTTI")

In [23]:
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])
#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

df1 = pd.read_csv('/content/drive/MyDrive/nucleare/STAGING_MARCO/PYTHON_TRADOTTI/combined_csv.csv')
df1.head(1)

,text_nonl_hash_men_http_num
0,"We never wanted to see you like this, Matthew 💔 you're great, you'll be stronger than before🎾 | |"


## Vader it

In [25]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


vader=[]

analyzer = SentimentIntensityAnalyzer()

for x in (df1.loc[:,'text_nonl_hash_men_http_num']):
  a = analyzer.polarity_scores(x)
  vader.append(a)

In [26]:
df1 = pd.DataFrame(vader)
df1.shape

(10000, 4)

In [27]:
print(df1)
df1.to_csv('/content/drive/MyDrive/nucleare/STAGING_MARCO/Vader_it.csv')

        neg    neu    pos  compound
0     0.095  0.492  0.412    0.8834
1     0.000  1.000  0.000    0.0000
2     0.000  1.000  0.000    0.0000
3     0.262  0.738  0.000   -0.4926
4     0.000  0.909  0.091    0.4588
...     ...    ...    ...       ...
9995  0.090  0.910  0.000   -0.4847
9996  0.000  1.000  0.000    0.0000
9997  0.274  0.726  0.000   -0.6007
9998  0.091  0.909  0.000   -0.2960
9999  0.059  0.722  0.219    0.7424

[10000 rows x 4 columns]


In [40]:
df1 = pd.read_csv('/content/drive/MyDrive/nucleare/STAGING_MARCO/Vader_it.csv')
df1.head(4)

,Unnamed: 0,neg,neu,pos,compound
0,0,0.095,0.492,0.412,0.8834
1,1,0.000,1.000,0.000,0.0000
2,2,0.000,1.000,0.000,0.0000
3,3,0.262,0.738,0.000,-0.4926


## MERGE DATASET 

In [29]:
result_merge_it = pd.concat([dfit, df1], axis=1).reindex(dfit.index)
result_merge_it.head(1)
result_merge_it.shape

In [31]:
result_merge_it.to_csv('/content/drive/MyDrive/nucleare/STAGING_MARCO/vader_merge_it.csv')

In [32]:
result_merge_en = pd.concat([dfen, df2], axis=1).reindex(dfen.index)
result_merge_en.head(1)
result_merge_en.shape

(49252, 41)

In [33]:
result_merge_en.to_csv('/content/drive/MyDrive/nucleare/STAGING_MARCO/vader_merge_en.csv')

In [37]:
prot_new_dataset = result_merge_it.append(result_merge_en)
prot_new_dataset.shape

(65237, 41)

In [38]:
prot_new_dataset.to_csv('/content/drive/MyDrive/nucleare/STAGING_MARCO/new_dataset.csv')

In [39]:
prot_new_dataset.head(1)

,id,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions,date_norm,time_zone,text_nonl,text_nonl_hash,text_nonl_hash_men,text_nonl_hash_men_http,text_nonl_hash_men_http_num,text_nonl_hash_men_http_num_simb,text_lang,text_lan_code,lang_confidence,prediction_reliable,Medvedev,Zverev,Berrettini,Hurkacz,Sinner,Djokovic,Tsitsipas,Rublev,Ruud,Norrie,ATP_Finals,topic,Unnamed: 0,neg,neu,pos,compound
0,1460367628070014991,MaricieloCuore,Mon Nov 15 22:02:29 +0000 2021,"Non avremmo mai voluto vederti così, Matteo 💔\n\nSei grande, tornerai più forte di prima 💪🎾\n\n#Berrettini | #NittoATPFinals | #EurosportTENNIS https://t.co/vj9Zv7tlQm",0,"['Berrettini', 'NittoATPFinals']",NaN,False,60,45,NaN,['Eurosport_IT'],2021-11-15 22:02:29,z+0000,"Non avremmo mai voluto vederti così, Matteo 💔 Sei grande, tornerai più forte di prima 💪🎾 #Berrettini | #NittoATPFinals | #EurosportTENNIS https://t.co/vj9Zv7tlQm","Non avremmo mai voluto vederti così, Matteo 💔 Sei grande, tornerai più forte di prima 💪🎾 | | https://t.co/vj9Zv7tlQm","Non avremmo mai voluto vederti così, Matteo 💔 Sei grande, tornerai più forte di prima 💪🎾 | | https://t.co/vj9Zv7tlQm","Non avremmo mai voluto vederti così, Matteo 💔 Sei grande, tornerai più forte di prima 💪🎾 | |","Non avremmo mai voluto vederti così, Matteo 💔 Sei grande, tornerai più forte di prima 💪🎾 | |",Non avremmo mai voluto vederti così Matteo Sei grande tornerai più forte di prima,Italian,it,98.0,True,0,0,1,0,0,0,0,0,0,0,1,['Berrettini'],0.0,0.095,0.492,0.412,0.8834


# 6. BoW



## 6.1 preprocessing our dataset: lematizzation, stopwords, ngrams

In [ ]:
import nltk ### libreria di linguistica (Stanford)
from nltk.stem import WordNetLemmatizer #### individua la radice linguistica delle parole: non tronca le parole come un stamming
nltk.download('wordnet') ## tassonomia lingua inglese
nltk.download('stopwords')

import warnings
from tqdm import tqdm
from pprint import pprint
from gensim.models.phrases import Phrases
from nltk.corpus import stopwords
pd.set_option('display.max_colwidth', None)

connectors = stopwords.words('english')
stop = stopwords.words('english')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
file_path = output_root_dir +'twitter_text_cleaned_lang_topic_det.csv'
df = pd.read_csv(file_path, encoding='utf-8')
df['text_cleaned'] = df['text_nonl_hash_men_http_num_simb']
print(df.isna().sum())
print(df.info())
print(df.columns)

In [ ]:
df = df[['id', 'user', 'date', 'text', 'text_cleaned','text_lan_code', 'lang_confidence', 'prediction_reliable', 'topic']]
print(df.isna().sum())
df.info()
df.head()

In [ ]:
print(f'original len: {len(df)}')

dff = df[df['text_lan_code'].isin(['en'])]
eng_len = len(dff)
print(f'english len: {eng_len}')

dff = dff[dff['prediction_reliable'] == True]
dff = dff[dff['lang_confidence'] > 96]

print(f'filtered len: {len(dff)}')
print(f'loss: {eng_len - len(dff)}')

print(dff.isna().sum())
print(dff.groupby(['text_lan_code', 'prediction_reliable']).size())
print(dff.groupby(['text_lan_code', 'lang_confidence']).size())

In [ ]:
df = dff.copy()

lemmatizer = WordNetLemmatizer()
df['text_cleaned'] = df['text_cleaned'].str.lower().str.replace('[^\w\s]',' ', regex=True) #type(text) series of string
df['text_cleaned'] = df['text_cleaned'].str.lower().str.replace('\s+', ' ', regex=True) ## rimuovi spazi multipli

raw_text = df[~df['text_cleaned'].isna()]['text_cleaned']

print(len(raw_text))

####pulizia: sostituisco tutti i caratteri speciali con uno spazio
text = raw_text.str.lower()
print(len(text))
print(text.head())

###TOKENIZATION
df['token'] = text.str.split() ##text: series of list, text[0] list of string, text[0][0] string 

###LEMMATIZATION
df['ltoken'] = df.token.apply(lambda x: [lemmatizer.lemmatize(sent) for sent in x]) #text: series of list, text[0] list of string (lemmatized), text[0][0] string

# print(f'text[0][0]-({type(df.ltoken[0][0])})')
# print(f'text[0]-({type(df.ltoken[0])})')
# print(f'text-({type(df.ltoken)})')

print(f'text_cleaned: {df.text_cleaned[0]}')
print(f'token       : {df.token[0]}')
print(f'ltoken      : {df.ltoken[0]}')

# stop.extend(['good', 'bad', 'dont', 'many', 'love', 'excellent', 'would', 'perfect', 'even', 'great','nice', 'amazing'])


In [ ]:
number = 1
print(f'text_cleaned: {df.text_cleaned.to_list()[number]}')
print(f'token       : {df.token.to_list()[number]}')
print(f'ltoken      : {df.ltoken.to_list()[number]}')

In [ ]:
print(df.isna().sum())
len_target = 1
field_name ='token' 
counter = df[df[field_name].str.len() < len_target ][field_name].count()
print(f'{field_name} with len < {len_target}: {counter}')

In [ ]:
my_stop_words = []
stop.extend(my_stop_words)

df['ltoken'] = df.ltoken.apply(lambda row: [item for item in row if item not in stop])

df['stopwords'] = df.ltoken.apply(lambda row: [item for item in row if item in stop])
df['nostopwords'] = df.ltoken.apply(lambda row: [item for item in row if item not in stop])

bigram  = Phrases(df.ltoken, min_count=5, threshold=0.2, common_terms=connectors) #gensim 3, ad esempio "and" 
df['bigrams_afternostop'] = df.ltoken.apply(lambda row: bigram[row])

ngram   = Phrases(df.bigrams_afternostop, min_count=5, threshold=0.2, common_terms=connectors) # gensim 3
df['ngrams_afternostop'] = df.bigrams_afternostop.apply(lambda row: ngram[row])

df.bigrams_afternostop = df.bigrams_afternostop.apply(lambda bigr: [item for item in bigr if item not in stop])
df.ngrams_afternostop = df.ngrams_afternostop.apply(lambda ngr: [item for item in ngr if item not in stop])

bigram  = Phrases(df.nostopwords, min_count=5, threshold=0.2, common_terms=connectors) #gensim 3, ad esempio "and" 
df['bigrams_beforenostop'] = df.nostopwords.apply(lambda row: bigram[row])

ngram   = Phrases(df.bigrams_beforenostop, min_count=5, threshold=0.2, common_terms=connectors) # gensim 3
df['ngrams_beforenostop'] = df.bigrams_beforenostop.apply(lambda row: ngram[row])

df['bigrams_afternostop_sent'] = df.bigrams_afternostop.apply(lambda row: ' '.join(row))
df['bigrams_beforenostop_sent'] = df.bigrams_beforenostop.apply(lambda row: ' '.join(row))
df['ngrams_afternostop_sent'] = df.ngrams_afternostop.apply(lambda row: ' '.join(row))
df['ngrams_beforenostop_sent'] = df.ngrams_beforenostop.apply(lambda row: ' '.join(row))

In [ ]:
# print(df.isna().sum())
len_target = 1
# 'bigrams_afternostop_sent' 
# 'bigrams_beforenostop_sent'
# 'ngrams_afternostop_sent'  
# 'ngrams_beforenostop_sent' 
field_name ='ngrams_beforenostop_sent' 
counter = df[df[field_name].str.len() < len_target ][field_name].count()
print(f'{field_name} with len < {len_target}: {counter}')

n = 10
print(f'df.bigrams_afternostop_sent[{n}] : {df.bigrams_afternostop_sent.to_list()[n]}')
print(f'df.bigrams_beforenostop_sent[{n}]: {df.bigrams_beforenostop_sent.to_list()[n]}')
print(f'df.ngrams_afternostop_sent[{n}]  : {df.ngrams_afternostop_sent.to_list()[n]}')
print(f'df.ngrams_beforenostop_sent[{n}] : {df.ngrams_beforenostop_sent.to_list()[n]}')

In [ ]:
df.columns

### 6.1.1 Saving

In [ ]:
df_saving = df.copy() #.replace('\n','', regex=True)
df_saving = df_saving[['id', 'user', 'date', 'text', 'text_cleaned','text_lan_code', 'lang_confidence', 'prediction_reliable', 'topic',
       'bigrams_afternostop_sent', 'bigrams_beforenostop_sent',
       'ngrams_afternostop_sent', 'ngrams_beforenostop_sent']]
saving_file_name = f'{sentiment_root_dir}ngrams_ourdataset_dataframe.csv'
df_saving.to_csv(saving_file_name, index=False, encoding='utf-8', sep =',' )
df_saving.isna().sum()

## 6.2 Preprocesing labelled dataset

In [ ]:
pd.set_option('display.max_colwidth', None)
file_path = st_tone_root_dir +'label_IT.csv'
df_open_label = pd.read_csv(file_path)
len(df_open_label)
df_open_label.columns

Index(['id', 'user', 'date', 'text', 'favorite_count', 'hashtags', 'location',
       'retweet', 'retweet_count', 'followers_count', 'in_reply_to_status_id',
       'user_mentions', 'date_norm', 'time_zone', 'text_nonl',
       'text_nonl_hash', 'text_nonl_hash_men', 'text_nonl_hash_men_http',
       'text_nonl_hash_men_http_num', 'text_nonl_hash_men_http_num_simb',
       'text_lang', 'text_lan_code', 'lang_confidence', 'prediction_reliable',
       'Medvedev', 'Zverev', 'Berrettini', 'Hurkacz', 'Sinner', 'Djokovic',
       'Tsitsipas', 'Rublev', 'Ruud', 'Norrie', 'ATP_Finals', 'topic',
       'sentiment'],
      dtype='object')

In [ ]:
df_open_label.groupby(['lang_confidence']).size()

lang_confidence
50.0      3
60.0      1
64.0      3
80.0      1
85.0      1
87.0     22
92.0      6
93.0      3
94.0      7
95.0     30
96.0    109
97.0     92
98.0    283
99.0    531
dtype: int64

In [ ]:
df_label = df_open_label.copy()
print(df_label.groupby('sentiment').size())
df_label.sentiment = df_label.sentiment.astype('int')
print(len(df_label))
df_label = df_label[df_label.sentiment < 2]
print(len(df_label))

sentiment
0.0     27
1.0    284
2.0    781
dtype: int64
1092
311


In [ ]:
import nltk ### libreria di linguistica (Stanford)
nltk.download('wordnet') ## tassonomia lingua inglese

from tqdm import tqdm
from nltk.stem import WordNetLemmatizer #### individua la radice linguistica delle parole: non tronca le parole come un stamming
lemmatizer = WordNetLemmatizer()

raw_text = df_label[~df_label['text_nonl_hash_men_http_num'].isna()].loc[:, 'text_nonl_hash_men_http_num']
print(len(raw_text))

####pulizia: sostituisco tutti i caratteri speciali con uno spazio
text = raw_text.str.lower().str.replace('[^\w\s]',' ', regex=True) #type(text) series of string
text = text.str.lower().str.replace('\s+', ' ', regex=True) ## rimuovi spazi multipli
# text = raw_text.str.replace('!"%&()?^,.-;:_', ' ', regex=False) ## rimuovi spazi multipli
print(len(text))

# ##TOKENIZATION
df_label['token'] = text.str.split() ##text: series of list, text[0] list of string, text[0][0] string

# ###LEMMATIZATION
df_label['ltoken'] = df_label.token.apply(lambda x: [lemmatizer.lemmatize(sent) for sent in x]) #text: series of list, text[0] list of string (lemmatized), text[0][0] string

print(f'text[0][0]-({type(df_label.token[3][0])})')
print(f'text[0]-({type(df_label.token[3])})')
print(f'text-({type(df_label.token)})')
print(f'text[0][0]-({(df_label.token[3][0])})')
print(f'text[0]-({(df_label.token[3])})')
# print(f'text-({(df.token)})')
print(df_label.isna().sum())


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
311
311
text[0][0]-(<class 'str'>)
text[0]-(<class 'list'>)
text-(<class 'pandas.core.series.Series'>)
text[0][0]-(zverev)
text[0]-(['zverev', 'mi', 'è', 'sempre', 'piaciuto', 'dopo', 'questo', 'gesto', 'ancora', 'di', 'più'])
id                                    0
user                                  0
date                                  0
text                                  0
favorite_count                        0
hashtags                              0
location                            140
retweet                               0
retweet_count                         0
followers_count                       0
in_reply_to_status_id               294
user_mentions                         0
date_norm                             0
time_zone                             0
text_nonl                             0
text_nonl_hash                        0
text_nonl_hash_men

In [ ]:
stop.extend(['good', 'bad', 'dont', 'many', 'love', 'excellent', 'would', 'perfect', 'even', 'great','nice', 'amazing'])
my_stop_words = []

stop.extend(my_stop_words)

df_label['ltoken'] = df_label.ltoken.apply(lambda row: [item for item in row if item not in stop])

# df['stopwords'] = df.ltoken.apply(lambda row: [item for item in row if item in stop])
# df['nostopwords'] = df.ltoken.apply(lambda row: [item for item in row if item not in stop])



In [ ]:
df_label['ltoken']

1                                                                                                                                                                                                         [zverev, mi, è, sempre, piaciuto, dopo, questo, gesto, ancora, di, più]
3                                                                                                                                                                                                         [zverev, mi, è, sempre, piaciuto, dopo, questo, gesto, ancora, di, più]
4                                                                                                                                                                                                                                     [lo, sport, dovrebbe, essere, sempre, così]
5                                                                                                                                                                                 

In [ ]:
number = 3
print(f'comment     : {raw_text.to_list()[number]}')
print(f'token       : {df_label.token.to_list()[number]}')
print(f'ltoken      : {df_label.ltoken.to_list()[number]}')

print(df_label.isna().sum())

len_target = 1
field_name ='ltoken' 

counter = df_label[df_label[field_name].str.len() < len_target ][field_name].count()
print(f'{field_name} with len < {len_target}: {counter}/{len(df_label)}')
df_label = df_label[df_label['ltoken'].str.len() > 0]

counter = df_label[df_label[field_name].str.len() < len_target ][field_name].count()
print(f'{field_name} with len < {len_target}: {counter}/{len(df_label)}')
df_label.isna().sum()

comment     : Due su tre sul veloce, , è un dio. Solo lo può fermare.
token       : ['due', 'su', 'tre', 'sul', 'veloce', 'è', 'un', 'dio', 'solo', 'lo', 'può', 'fermare']
ltoken      : ['due', 'su', 'tre', 'sul', 'veloce', 'è', 'un', 'dio', 'solo', 'lo', 'può', 'fermare']
id                                    0
user                                  0
date                                  0
text                                  0
favorite_count                        0
hashtags                              0
location                            140
retweet                               0
retweet_count                         0
followers_count                       0
in_reply_to_status_id               294
user_mentions                         0
date_norm                             0
time_zone                             0
text_nonl                             0
text_nonl_hash                        0
text_nonl_hash_men                    0
text_nonl_hash_men_http               0
text_n

id                                    0
user                                  0
date                                  0
text                                  0
favorite_count                        0
hashtags                              0
location                            140
retweet                               0
retweet_count                         0
followers_count                       0
in_reply_to_status_id               294
user_mentions                         0
date_norm                             0
time_zone                             0
text_nonl                             0
text_nonl_hash                        0
text_nonl_hash_men                    0
text_nonl_hash_men_http               0
text_nonl_hash_men_http_num           0
text_nonl_hash_men_http_num_simb      0
text_lang                             0
text_lan_code                         0
lang_confidence                       0
prediction_reliable                   0
Medvedev                              0


In [ ]:

bigram  = Phrases(df_label.ltoken, min_count=5, threshold=0.2, common_terms=connectors) #gensim 3, ad esempio "and" 
df_label['bigrams_afternostop'] = df_label.ltoken.apply(lambda row: bigram[row])

ngram   = Phrases(df_label.bigrams_afternostop, min_count=5, threshold=0.2, common_terms=connectors) # gensim 3
df_label['ngrams_afternostop'] = df_label.bigrams_afternostop.apply(lambda row: ngram[row])

# df_label.bigrams_afternostop = df_label.bigrams_afternostop.apply(lambda bigr: [item for item in bigr if item not in stop])
df_label.ngrams_afternostop = df_label.ngrams_afternostop.apply(lambda ngr: [item for item in ngr if item not in stop])

# bigram  = Phrases(df_label.nostopwords, min_count=20, threshold=0.2, common_terms=connectors) #gensim 3, ad esempio "and" 
# df_label['bigrams_beforenostop'] = df_label.nostopwords.apply(lambda row: bigram[row])

# ngram   = Phrases(df_label.bigrams_beforenostop, min_count=20, threshold=0.2, common_terms=connectors) # gensim 3
# df_label['ngrams_beforenostop'] = df_label.bigrams_beforenostop.apply(lambda row: ngram[row])


In [ ]:

df_label['bigrams_afternostop_sent'] = df_label.bigrams_afternostop.apply(lambda row: ' '.join(row))
df_label['ngrams_afternostop_sent'] = df_label.ngrams_afternostop.apply(lambda row: ' '.join(row))

# df_label['bigrams_beforenostop_sent'] = df_label.bigrams_beforenostop.apply(lambda row: ' '.join(row))
# df_label['ngrams_beforenostop_sent'] = df_label.ngrams_beforenostop.apply(lambda row: ' '.join(row))

In [ ]:

# df_label.bigrams_afternostop_sent = df_label.bigrams_afternostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()
# df_label.bigrams_beforenostop_sent = df_label.bigrams_beforenostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()
# df_label.ngrams_afternostop_sent = df_label.ngrams_afternostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()
# df_label.ngrams_beforenostop_sent = df_label.ngrams_beforenostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()

print(len(df_label))
df_label = df_label[~(df_label.bigrams_afternostop_sent.str.len() == 0)]
# df_label = df_label[~(df_label.bigrams_beforenostop_sent.str.len() == 0)]
df_label = df_label[~(df_label.ngrams_afternostop_sent.str.len() == 0)]
# df_label = df_label[~(df_label.ngrams_beforenostop_sent.str.len() == 0)]
print(len(df_label))

311
311


In [ ]:


# print(df_label.isna().sum())
len_target = 1
for field_name in ['token', 'ltoken'
              ,'bigrams_afternostop_sent' 
              # ,'bigrams_beforenostop_sent'
              ,'ngrams_afternostop_sent'  
              # ,'ngrams_beforenostop_sent'
              ]: 
  counter = df_label[df_label[field_name].str.len() < len_target][field_name].count()
  print(f'{field_name} with len < {len_target}: {counter}')

n = 10
print(f'raw_text[{n}] : {raw_text.to_list()[n]}')

print(f'df_label.bigrams_afternostop_sent[{n}] : {df_label.bigrams_afternostop_sent.to_list()[n]}')
# print(f'df_label.bigrams_beforenostop_sent[{n}]: {df_label.bigrams_beforenostop_sent.to_list()[n]}')
print(f'df_label.ngrams_afternostop_sent[{n}]  : {df_label.ngrams_afternostop_sent.to_list()[n]}')
# print(f'df_label.ngrams_beforenostop_sent[{n}] : {df_label.ngrams_beforenostop_sent.to_list()[n]}')

token with len < 1: 0
ltoken with len < 1: 0
bigrams_afternostop_sent with len < 1: 0
ngrams_afternostop_sent with len < 1: 0
raw_text[10] : vince il secondo set su È già capolavoro ragazzo! 
df_label.bigrams_afternostop_sent[10] : vince il_secondo set su è già capolavoro ragazzo
df_label.ngrams_afternostop_sent[10]  : vince il_secondo_set su è già capolavoro ragazzo


In [ ]:
df_label_saving = df_label.copy()

df_label_saving['cleaned_text'] = df_label_saving['text_nonl_hash_men_http_num']

df_label_saving = df_label_saving.loc[:, ['id', 'user', 'date', 'text', 'favorite_count', 'hashtags', 'location',
       'retweet', 'retweet_count', 'followers_count', 'in_reply_to_status_id',
       'user_mentions', 'date_norm', 'time_zone', 'cleaned_text', 'lang_confidence', 'prediction_reliable',
       'Medvedev', 'Zverev', 'Berrettini', 'Hurkacz', 'Sinner', 'Djokovic',
       'Tsitsipas', 'Rublev', 'Ruud', 'Norrie', 'ATP_Finals', 'topic',
       'token', 'ltoken', 'bigrams_afternostop',
       'ngrams_afternostop', 'bigrams_afternostop_sent',
       'ngrams_afternostop_sent', 'sentiment']]
df_label_saving.columns

Index(['id', 'user', 'date', 'text', 'favorite_count', 'hashtags', 'location',
       'retweet', 'retweet_count', 'followers_count', 'in_reply_to_status_id',
       'user_mentions', 'date_norm', 'time_zone', 'cleaned_text',
       'lang_confidence', 'prediction_reliable', 'Medvedev', 'Zverev',
       'Berrettini', 'Hurkacz', 'Sinner', 'Djokovic', 'Tsitsipas', 'Rublev',
       'Ruud', 'Norrie', 'ATP_Finals', 'topic', 'token', 'ltoken',
       'bigrams_afternostop', 'ngrams_afternostop', 'bigrams_afternostop_sent',
       'ngrams_afternostop_sent', 'sentiment'],
      dtype='object')

In [ ]:
saving_file_name = f'{st_tone_root_dir}dflabaled_ready4_bow.csv'
df_label_saving.to_csv(saving_file_name, index=False, encoding='utf-8', sep =',' )
df_label_saving.isna().sum()

id                            0
user                          0
date                          0
text                          0
favorite_count                0
hashtags                      0
location                    140
retweet                       0
retweet_count                 0
followers_count               0
in_reply_to_status_id       294
user_mentions                 0
date_norm                     0
time_zone                     0
cleaned_text                  0
lang_confidence               0
prediction_reliable           0
Medvedev                      0
Zverev                        0
Berrettini                    0
Hurkacz                       0
Sinner                        0
Djokovic                      0
Tsitsipas                     0
Rublev                        0
Ruud                          0
Norrie                        0
ATP_Finals                    0
topic                         0
token                         0
ltoken                        0
bigrams_

## 6.3 BOW

### 6.3.1 Fit tennis dictionary and save it

In [ ]:
opening_file_name = f'{sentiment_root_dir}ngrams_ourdataset_dataframe.csv'
df_open_full = pd.read_csv(opening_file_name, encoding='utf-8', sep =',' )
df_open_full.isna().sum()

id                           0
user                         0
date                         0
text                         0
text_cleaned                 0
text_lan_code                0
lang_confidence              0
prediction_reliable          0
topic                        0
bigrams_afternostop_sent     0
bigrams_beforenostop_sent    0
ngrams_afternostop_sent      0
ngrams_beforenostop_sent     0
dtype: int64

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
import pickle

df_full = df_open_full.copy()

vectorizer = CountVectorizer(ngram_range=(1, 1), max_features=1000)
# vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=1000)
X = vectorizer.fit(df_full['ngrams_afternostop_sent'].to_list())
feature_names = vectorizer.get_feature_names()
# print(feature_names)

model_filename = f"{models_root_dir}our_bow_dict_model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(vectorizer, file)


In [ ]:
print(X)
print(feature_names[0:100])

CountVectorizer(max_features=1000)
['able', 'absolutely', 'absolutely_pathetic_schedule_tournament', 'account', 'ace', 'action', 'actually', 'add', 'age_player_atp_final', 'agree', 'ahead', 'ahead_debut_evening_ha', 'al', 'alert_new_high_roller', 'alex_zverev', 'alexander', 'alexander_second_defeat_medvedev', 'alexander_zverev', 'allez_amp_champion_second', 'almost', 'already', 'also', 'alternate', 'always', 'always_applauding_opponent_hit', 'amazing', 'amp', 'amp_defeat_ram_salisbury', 'amp_grande_ibra_uvek', 'amp_stay_unblemished', 'amp_two_time_champion', 'ana', 'andre_agassi_alexander_zverev', 'andrei_rublev', 'andrey', 'andrey_rublev', 'andrey_rublev_beat_stefanos', 'anime_murder_ok_john', 'another', 'answer', 'anyone', 'anyone_offends_know_far', 'anything', 'anyway', 'apart_djokovic_djokovic_least', 'around', 'ask', 'asked', 'atp', 'atp_final', 'atp_final_defending_champ', 'atp_final_sunday_pm', 'atp_tour', 'atp_tour_season', 'atp_world_tour_final', 'attack', 'australian_open_rol

### 6.3.2 Labelled csv


In [ ]:
df_open_label = pd.read_csv(f'{st_tone_root_dir}dflabaled_ready4_bow.csv', encoding='utf-8', sep =',', header=0 )

# df.bigrams_afternostop_sent = df.bigrams_afternostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()
# df.bigrams_beforenostop_sent = df.bigrams_beforenostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()
# df.ngrams_afternostop_sent = df.ngrams_afternostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()
# df.ngrams_beforenostop_sent = df.ngrams_beforenostop_sent.str.replace('[^\w\s]',' ').str.replace('\s+', ' ', regex=True).str.strip()

# print(len(df_open))
# df_open = df_open[~(df_open.bigrams_afternostop_sent.str.len() == 0)]
# df_open = df_open[~(df_open.bigrams_beforenostop_sent.str.len() == 0)]
# df_open = df_open[~(df_open.ngrams_afternostop_sent.str.len() == 0)]
# df_open = df_open[~(df_open.ngrams_beforenostop_sent.str.len() == 0)]
# print(len(df_open))

print(df_open_label.isna().sum())
df_open_label.head(2)

id                            0
user                          0
date                          0
text                          0
favorite_count                0
hashtags                      0
location                    140
retweet                       0
retweet_count                 0
followers_count               0
in_reply_to_status_id       294
user_mentions                 0
date_norm                     0
time_zone                     0
cleaned_text                  0
lang_confidence               0
prediction_reliable           0
Medvedev                      0
Zverev                        0
Berrettini                    0
Hurkacz                       0
Sinner                        0
Djokovic                      0
Tsitsipas                     0
Rublev                        0
Ruud                          0
Norrie                        0
ATP_Finals                    0
topic                         0
token                         0
ltoken                        0
bigrams_

,id,user,date,text,favorite_count,hashtags,location,retweet,retweet_count,followers_count,in_reply_to_status_id,user_mentions,date_norm,time_zone,cleaned_text,lang_confidence,prediction_reliable,Medvedev,Zverev,Berrettini,Hurkacz,Sinner,Djokovic,Tsitsipas,Rublev,Ruud,Norrie,ATP_Finals,topic,token,ltoken,bigrams_afternostop,ngrams_afternostop,bigrams_afternostop_sent,ngrams_afternostop_sent,sentiment
0,1460007859798827012,illeaoiano,Sun Nov 14 22:12:54 +0000 2021,"zverev mi è sempre piaciuto, dopo questo gesto ancora di più https://t.co/9ElOVRFyVM",0,[],NaN,False,13,53,NaN,['acm95fede'],2021-11-14 22:12:54,z+0000,"zverev mi è sempre piaciuto, dopo questo gesto ancora di più",98.0,True,0,0,0,0,0,0,0,0,0,0,0,[],"['zverev', 'mi', 'è', 'sempre', 'piaciuto', 'dopo', 'questo', 'gesto', 'ancora', 'di', 'più']","['zverev', 'mi', 'è', 'sempre', 'piaciuto', 'dopo', 'questo', 'gesto', 'ancora', 'di', 'più']","['zverev_mi', 'è_sempre', 'piaciuto_dopo', 'questo_gesto', 'ancora_di', 'più']","['zverev_mi_è_sempre', 'piaciuto_dopo_questo_gesto', 'ancora_di_più']",zverev_mi è_sempre piaciuto_dopo questo_gesto ancora_di più,zverev_mi_è_sempre piaciuto_dopo_questo_gesto ancora_di_più,1
1,1460007472245137412,alessia__Z,Sun Nov 14 22:11:21 +0000 2021,"zverev mi è sempre piaciuto, dopo questo gesto ancora di più https://t.co/9ElOVRFyVM",0,[],Milanello,False,13,1505,NaN,['acm95fede'],2021-11-14 22:11:21,z+0000,"zverev mi è sempre piaciuto, dopo questo gesto ancora di più",98.0,True,0,0,0,0,0,0,0,0,0,0,0,[],"['zverev', 'mi', 'è', 'sempre', 'piaciuto', 'dopo', 'questo', 'gesto', 'ancora', 'di', 'più']","['zverev', 'mi', 'è', 'sempre', 'piaciuto', 'dopo', 'questo', 'gesto', 'ancora', 'di', 'più']","['zverev_mi', 'è_sempre', 'piaciuto_dopo', 'questo_gesto', 'ancora_di', 'più']","['zverev_mi_è_sempre', 'piaciuto_dopo_questo_gesto', 'ancora_di_più']",zverev_mi è_sempre piaciuto_dopo questo_gesto ancora_di più,zverev_mi_è_sempre piaciuto_dopo_questo_gesto ancora_di_più,1


In [ ]:
df_open_label.shape

(1092, 37)

In [ ]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split

df = df_open_label.copy()
# number_of_row = 1000
# # number_of_row = len(df['y'] == 1])
# df = df[df['y'] == 1].sample(number_of_row, random_state= 123).append(df[df['y'] == 0].sample(number_of_row, random_state= 123))
# print(f'reduced df len: {len(df)}')
# # 'comment', 'rating', 'bigrams_afternostop_sent',
# #        'bigrams_beforenostop_sent', 'ngrams_afternostop_sent',
# #        'ngrams_beforenostop_sent', 'y'

# choosen_column_2vec = 'ngrams_afternostop_sent'

# vectorizer = CountVectorizer(ngram_range=(1, 1), max_features=1000)
# # vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=1000)

# model_filename = f"{models_root_dir}our_bow_dict_model.pkl"
# with open(model_filename, 'rb') as file:
#     vectorizer = pickle.load(file)

X = vectorizer.transform(df['ngrams_afternostop_sent'].to_list())
# # feature_names = vectorizer.get_feature_names()
# # print(feature_names)

X = X.toarray()
X = np.array(X)
y = np.array(df['sentiment'].to_list())

x_train, x_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.1, random_state=42)


In [ ]:
all_ngrams = []
for ngram_list in df.ngrams_afternostop_sent.to_list():
  all_ngrams.extend(ngram_list.split())

from collections import Counter
counter = Counter(all_ngrams)
counter.most_common(100)

In [ ]:
feature_names = vectorizer.get_feature_names()
feature_names[:100]

In [ ]:
from tqdm import tqdm
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import accuracy_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import pickle

#################################################################################à
tree_model = tree.DecisionTreeClassifier(max_leaf_nodes=10, max_depth=5)
#################################################################################à
tree_model.fit(x_train, y_train)

# Save to file in the current working directory
model_filename = f"{models_root_dir}our_bow_dict_model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(tree_model, file)
# Load from file
with open(model_filename, 'rb') as file:
    tree_model = pickle.load(file)

predicted = tree_model.predict(x_test)
print(f"""tree_model: 
{classification_report(y_test, predicted)}""")

#################################################################################à
logit_model = LogisticRegression(class_weight=None)
#################################################################################à
logit_model.fit(x_train, y_train)
# Save to file in the current working directory
model_filename = f"{models_root_dir}basic_logit_model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(logit_model, file)
# Load from file
with open(model_filename, 'rb') as file:
    logit_model = pickle.load(file)

predicted = logit_model.predict(x_test)
print(f"""logit_model: 
{classification_report(y_test, predicted)}""")

#################################################################################à
random_model = RandomForestClassifier()
#################################################################################à
random_model.fit(x_train, y_train)
# Save to file in the current working directory
model_filename = f"{models_root_dir}basic_random_model.pkl"
with open(model_filename, 'wb') as file:
    pickle.dump(random_model, file)
# Load from file
with open(model_filename, 'rb') as file:
    random_model = pickle.load(file)

predicted = random_model.predict(x_test)
print(f"""random_model: 
{classification_report(y_test, predicted)}""")


tree_model: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.90      0.97      0.93        29

    accuracy                           0.88        32
   macro avg       0.45      0.48      0.47        32
weighted avg       0.82      0.88      0.85        32

logit_model: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.91      1.00      0.95        29

    accuracy                           0.91        32
   macro avg       0.45      0.50      0.48        32
weighted avg       0.82      0.91      0.86        32

random_model: 
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.90      0.97      0.93        29

    accuracy                           0.88        32
   macro avg       0.45      0.48      0.47        32
weighted avg       0.82      0.8

# 7. Word embeddings
TONE'

# 8. Lime

# 9. SocialNetwork Analisys